#Adicionando TinyDb


In [ ]:
from tinydb import TinyDB, Query
from pymongo import MongoClient
import datetime
import os
import pandas as pd

In [ ]:
db = TinyDB(os.path.join('DB', 'estrategys.json'))

In [ ]:
listaDeArquivosDeEstrategias = os.listdir("estrategy results")

In [ ]:
doc1 = pd.read_csv(os.path.join("estrategy results",
                   listaDeArquivosDeEstrategias[0]), sep=';')

In [ ]:
def stringToDf(string):
    df = pd.DataFrame([x.split(';') for x in string.split('\n')])
    df.columns = ['variable']
    df['value'] = df['variable'].apply(lambda x: x.split('   ')[-1])
    df['variable'] = df['variable'].apply(lambda x: x.split('   ')[0])
    return df


In [ ]:
def iterate_df(df):
    maiorValor = []
    for i in range(len(df)):
        maiorValor.append(df.iloc[i].to_dict())
    return maiorValor


In [ ]:
iterado = iterate_df(doc1)
arrey = listaDeArquivosDeEstrategias[0].split("_")
final = stringToDf(iterado[0]['output'])

In [ ]:
def dataframeParaDicionario(dicionario, arrey_do_nome_da_estrategia_separado):
    try:
        listaDeResultados = {}
        for item in stringToDf(dicionario['output']).values.tolist():
            item1 = item[0].replace('_', '').replace('#', '').strip()
            item2 = item[1].replace('\r', '').replace(
                '\n', '').replace('...', '').strip()

            if item1 == "Trades" or item1 == "Exposure Time [%]" or item1 == "Exposure Time [%]" or item1 == "Return [%]" or item1 == "Buy & Hold Return [%]" or item1 == "Return (Ann.) [%]" or item1 == 'Volatility (Ann.) [%]' or item1 == 'Equity Final [$]' or item1 == 'Equity Peak [$]' or item1 == 'Sharpe Ratio' or item1 == 'Sortino Ratio' or item1 == 'Calmar Ratio' or item1 == 'Max. Drawdown [%]' or item1 == 'Avg. Drawdown [%]' or item1 == 'Win Rate [%]' or item1 == 'Best Trade [%]' or item1 == 'Worst Trade [%]' or item1 == 'Expectancy [%]' or item1 == 'SQN' or item1 == 'Profit Factor' or item1 == 'Avg. Trade [%]':
                item2 = float(item2)
            listaDeResultados.__setitem__(item1, item2)
        resultado = {"ticket": arrey_do_nome_da_estrategia_separado[0], "strategy": {
            "name": arrey_do_nome_da_estrategia_separado[2], "timeFrame": arrey_do_nome_da_estrategia_separado[1], "params": {"SMA1": float(dicionario['sma1']), "SMA2": float(dicionario['sma2']), }, },  "results": listaDeResultados}
        return resultado
    except Exception as e:
        print(e)


In [ ]:
def adicionarAListaDeResultado(lista, resultado):
    # se o resultado não estiver na lista, adiciona
    if resultado not in lista:
        lista.append(resultado)
    return lista

In [ ]:
listaDeArquivosDeEstrategias = os.listdir("estrategy results")
index = 0
for itemName in listaDeArquivosDeEstrategias:
    doc1 = pd.read_csv(os.path.join("estrategy results", itemName), sep=';')
    resultado = iterate_df(doc1)
    arrey_do_nome_da_estrategia_separado = itemName.split("_")
    print('inserindo: ', arrey_do_nome_da_estrategia_separado, 'index: ', index)
    maxReturn = None
    maxSharpeRatio = None
    maxSortinoRatio = None
    maxCalmarRatio = None
    WinRate = None
    maxProfitFactor = None
    maxExpectancy = None
    maxSQN = None
    finalList = []
    for dataframe in resultado:        
        dictionary = dataframeParaDicionario(dataframe,arrey_do_nome_da_estrategia_separado)
        if maxReturn == None or dictionary['results']['Return [%]'] > maxReturn['results']['Return [%]']:
            maxReturn = dictionary
        if maxSharpeRatio == None or dictionary['results']['Sharpe Ratio'] > maxSharpeRatio['results']['Sharpe Ratio']:
            maxSharpeRatio = dictionary
        if maxSortinoRatio == None or dictionary['results']['Sortino Ratio'] > maxSortinoRatio['results']['Sortino Ratio']:
            maxSortinoRatio = dictionary
        if maxCalmarRatio == None or dictionary['results']['Calmar Ratio'] > maxCalmarRatio['results']['Calmar Ratio']:
            maxCalmarRatio = dictionary
        if WinRate == None or dictionary['results']['Win Rate [%]'] > WinRate['results']['Win Rate [%]']:
            WinRate = dictionary
        if maxProfitFactor == None or dictionary['results']['Profit Factor'] > maxProfitFactor['results']['Profit Factor']:
            maxProfitFactor = dictionary
        if maxExpectancy == None or dictionary['results']['Expectancy [%]'] > maxExpectancy['results']['Expectancy [%]']:
            maxExpectancy = dictionary
        if maxSQN == None or dictionary['results']['SQN'] > maxSQN['results']['SQN']:
            maxSQN = dictionary
    finalList = adicionarAListaDeResultado(finalList, maxReturn)
    finalList = adicionarAListaDeResultado(finalList, maxSharpeRatio)
    finalList = adicionarAListaDeResultado(finalList, maxSortinoRatio)
    finalList = adicionarAListaDeResultado(finalList, maxCalmarRatio)
    finalList = adicionarAListaDeResultado(finalList, WinRate)
    finalList = adicionarAListaDeResultado(finalList, maxProfitFactor)
    finalList = adicionarAListaDeResultado(finalList, maxExpectancy)
    finalList = adicionarAListaDeResultado(finalList, maxSQN)
    print('total de itens: ',len(finalList))
    table = db.table('estrategys')
    if finalList != [None]:
        table.insert_multiple(finalList)    
    print('inserção completa.')
    index = index + 1